# Training Model

#### Importing liabraries

In [1]:
import pickle
import music21
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


#### Importing notes

In [4]:
with open('data/notes.pickle', 'rb') as f:
    notes = pickle.load(f)

pitchNames = set (notes)
nNotes = len (pitchNames)
pitchNames = sorted (pitchNames)

#### Encoding notes

In [5]:
noteCoding = {}

code = 0
for i in pitchNames :
    noteCoding [i] = code
    code += 1

#### One hot encoding

In [6]:
sequence = 100
nInput = []
nOutput = []

for i in range (0, len (notes) - sequence, 1) : 
    
    sIn = notes [i : i + sequence]
    sOut = notes [i + sequence]
    nInput.append ([noteCoding [c]for c in sIn])
    nOutput.append (noteCoding [sOut])
    
nInput

[[342,
  342,
  291,
  342,
  355,
  349,
  343,
  342,
  355,
  354,
  325,
  348,
  342,
  344,
  356,
  350,
  342,
  292,
  356,
  350,
  342,
  292,
  313,
  292,
  342,
  327,
  304,
  345,
  311,
  342,
  342,
  291,
  342,
  355,
  349,
  343,
  342,
  355,
  354,
  325,
  348,
  342,
  344,
  356,
  350,
  342,
  292,
  356,
  350,
  342,
  292,
  313,
  292,
  342,
  327,
  324,
  345,
  330,
  341,
  341,
  291,
  341,
  355,
  349,
  289,
  343,
  355,
  310,
  325,
  341,
  344,
  356,
  350,
  341,
  292,
  356,
  350,
  341,
  292,
  313,
  292,
  341,
  327,
  303,
  345,
  310,
  341,
  341,
  291,
  341,
  355,
  349,
  289,
  343,
  355,
  310,
  325,
  341,
  344,
  356],
 [342,
  291,
  342,
  355,
  349,
  343,
  342,
  355,
  354,
  325,
  348,
  342,
  344,
  356,
  350,
  342,
  292,
  356,
  350,
  342,
  292,
  313,
  292,
  342,
  327,
  304,
  345,
  311,
  342,
  342,
  291,
  342,
  355,
  349,
  343,
  342,
  355,
  354,
  325,
  348,
  342,
  344,
  356

In [0]:
nInput = np.reshape (nInput, (len (nInput), sequence, 1))
nInput = nInput / float (nNotes)

In [0]:
nOutput = np_utils.to_categorical(nOutput)

#### Creating model

In [7]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(nInput.shape[1], nInput.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.25))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.25))
model.add(Dense(nNotes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary ()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

#### Saving weights

In [0]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

#### Training model

In [0]:
model.fit(nInput, nOutput, epochs = 100, batch_size = 64, callbacks = callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
57077/57077 [==============================] - 379s 7ms/step - loss: 4.7555
Epoch 2/100
57077/57077 [==============================] - 371s 7ms/step - loss: 4.7137
Epoch 3/100
57077/57077 [==============================] - 375s 7ms/step - loss: 5.2695
Epoch 4/100
57077/57077 [==============================] - 368s 6ms/step - loss: 4.7081
Epoch 5/100
57077/57077 [==============================] - 373s 7ms/step - loss: 4.7171
Epoch 6/100
57077/57077 [==============================] - 371s 7ms/step - loss: 4.7080
Epoch 7/100
57077/57077 [==============================] - 371s 7ms/step - loss: 4.7081
Epoch 8/100
57077/57077 [==============================] - 372s 7ms/step - loss: 4.7063
Epoch 9/100
57077/57077 [==============================] - 374s 7ms/step - loss: 4.7420
Epoch 10/100
57077/57077 [==============================] - 371s 7ms/step - loss: 4.6912
Epoch 11/100
57077/57077 [==============================] - 375s 7ms/st

#### Model trained